# Testing the Integrated Simulation Platform
This notebook contains codes to test the various functionalities of the integrated simulation platform. You can call any of the modules from the package here. 

In [1]:
#Go to the main directory
#!pip install --editable .
#!pip install pandas
# !pip install wntr
# !pip install pandapower
# !pip install sklearn

#to find requirements
#depfinder -y .

In [2]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Import required packages

In [3]:
from pathlib import Path
from infrarisk.src.network_recovery import *
import infrarisk.src.simulation as simulation
from infrarisk.src.network_sim_models.integrated_network import *
import infrarisk.src.recovery_strategies as strategies

from infrarisk.src.network_sim_models.interdependencies import *

from infrarisk.src.optimizer import *
import infrarisk.src.plots as model_plots

#import random

## Create an IntegratedNetwork object

In [4]:
micropolis_network = IntegratedNetwork(name = 'Micropolis')

### Load the three infrastructure models: Water, Power and Transportation

Three different models are used:
- Water distribution network using **wntr** package
- Power systems using **pandapower** package
- Transportation network using static traffic assignment package developed by Dr. Stephen Boyles (University of Texas at Austin)

In [5]:
MAIN_DIR = Path('..')
mesh_level = 'low'
network_dir= 'infrarisk/data/networks/micropolis'
water_folder = MAIN_DIR/f'{network_dir}/water/{mesh_level}'
power_folder = MAIN_DIR/f'{network_dir}/power/{mesh_level}'
transp_folder = MAIN_DIR/f'{network_dir}/transportation/{mesh_level}'

# load all infrastructure networks
micropolis_network.load_networks(water_folder, 
                                 power_folder, 
                                 transp_folder, 
                                 power_sim_type = '1ph', 
                                 water_sim_type = "PDA")

C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\wntr\epanet\io.py:2173: UserWarning: Not all curves were used in "..\infrarisk\data\networks\micropolis\water\low/water.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


Water network successfully loaded from ..\infrarisk\data\networks\micropolis\water\low/water.inp. The analysis type is set to PDA.
initial simulation duration: 60s; hydraulic time step: 60s; pattern time step: 3600s

Power system successfully loaded from ..\infrarisk\data\networks\micropolis\power\low\power.json. Single phase power flow simulation will be used.

Transportation network successfully loaded from ..\infrarisk\data\networks\micropolis\transportation\low. Static traffic assignment method will be used to calculate travel times.


### Create a Networkx graph of the integrated infrastructure network.

In [1]:
micropolis_network.generate_integrated_graph()

NameError: name 'micropolis_network' is not defined

### Build interdependencies

Three types of dependencies:
- Power - Water dependencies (eg.: water pump on electric motor, generator on reservoir)
- Power - Transportation dependencies (eg.: road access to power system components for M&R)
- Water - Transportation dependencies (eg.: road access to water network components for M&R)

The dependencies are referenced using two tables in the model.
- **wp_table** for water - power dependencies
- **access_table** for transportation dependencies

In [7]:
dependency_file = MAIN_DIR/f"{network_dir}/dependencies.csv"
micropolis_network.generate_dependency_table(dependency_file = dependency_file)
micropolis_network.dependency_table.wp_table

,water_id,power_id,water_type,power_type
0,W_WP1,P_MP1,Pump,Motor
1,W_WP2,P_MP1,Pump,Motor
2,W_WP3,P_MP1,Pump,Motor
3,W_WP4,P_MP1,Pump,Motor
4,W_WP5,P_MP2,Pump,Motor
5,W_WP6,P_MP2,Pump,Motor
6,W_WP7,P_MP2,Pump,Motor
7,W_WP8,P_MP3,Pump,Motor


In [8]:
micropolis_network.dependency_table.access_table.head()

,origin_id,transp_id,origin_cat,origin_type,access_dist
0,P_B0,T_J57,power,Bus,84.79
1,P_B1,T_J57,power,Bus,16.96
2,P_B2,T_J57,power,Bus,178.04
3,P_B3,T_J56,power,Bus,27.57
4,P_B4,T_J56,power,Bus,170.80


### Set failed components

In [9]:
scenario_folder = "scenarios/track0/low"
scenario_file = MAIN_DIR/f"{network_dir}/{scenario_folder}/disruption_file.csv"

micropolis_network.set_disrupted_components(scenario_file=scenario_file)
micropolis_network.get_disrupted_components()

['W_PMA687',
 'W_PMA855',
 'W_PMA856',
 'W_PMA834',
 'W_PMA833',
 'W_PMA737',
 'W_PMA770',
 'P_L377',
 'P_L336',
 'P_L345',
 'T_L37',
 'T_L168']

In [10]:
# micropolis_network.pipe_leak_node_generator()

### Deploy recovery crews

In [10]:
micropolis_network.deploy_crews(
    init_power_crew_locs=['T_J8', 'T_J8', 'T_J8', 'T_J8'], 
    init_water_crew_locs=['T_J8', 'T_J8', 'T_J8', 'T_J8'],
    init_transpo_crew_locs= ['T_J8', 'T_J8', 'T_J8', 'T_J8']
    )

Power repair crews successfully deployed.
Water repair crews successfully deployed.
Transportation repair crews successfully deployed.


## Simulation of interdependent effects using a test scenario
### (a) Create NetworkRecovery

In [11]:
network_recovery = NetworkRecovery(micropolis_network, sim_step=60, pipe_close_policy="repair")

### (b) Create a simulation object

In [12]:
sim_step = (
    micropolis_network.wn.options.time.hydraulic_timestep
)  # initial_sim_step which will be updated during the simulation
bf_simulation = simulation.NetworkSimulation(network_recovery, sim_step)

### (c) Generation of random repair order

In [13]:
capacity_strategy = strategies.HandlingCapacityStrategy(micropolis_network)
capacity_strategy.set_repair_order()
repair_order = capacity_strategy.get_repair_order()
# import geopandas as gpd
# zones_shp = '../infrarisk/data/networks/micropolis/gis/micropolis_zones.shp'
# zone_strategy = strategies.ZoneBasedStrategy(micropolis_network, zones_shp )
# zone_strategy.set_repair_order()
# repair_order = zone_strategy.get_repair_order()

import os
if not os.path.exists(MAIN_DIR/f"{network_dir}/{scenario_folder}/capacity"):
    os.makedirs(MAIN_DIR/f"{network_dir}/{scenario_folder}/capacity")

In [14]:
#Generate a random repair order
# repair_order = network_recovery.network.get_disrupted_components()
# repair_orders = pd.read_csv(MAIN_DIR/f"{network_dir}/{scenario_folder}/repair_strategies.csv")
# repair_order = list(repair_orders["capacity"])
#random.shuffle(repair_order)
print('Current repair order is {}'.format(repair_order))

Current repair order is ['T_L37', 'T_L168', 'P_L377', 'P_L336', 'P_L345', 'W_PMA834', 'W_PMA833', 'W_PMA687', 'W_PMA737', 'W_PMA855', 'W_PMA856', 'W_PMA770']


### (d) Generation of event tables

In [15]:
bf_simulation.network_recovery.schedule_recovery(repair_order)

Repair T_L37: The transpo crew 1 is at T_J8 at t = 100.0 minutes. It takes 15 minutes to reach nearest node T_J35, the nearest transportation node from T_L37.
Repair T_L168: The transpo crew 2 is at T_J8 at t = 100.0 minutes. It takes 19 minutes to reach nearest node T_J49, the nearest transportation node from T_L168.
Repair P_L377: The power crew 1 is at T_J8 at t = 100.0 minutes. It takes 21 minutes to reach nearest node T_J50,  the nearest transportation node from P_L377.
Repair P_L336: The power crew 2 is at T_J8 at t = 100.0 minutes. It takes 18 minutes to reach nearest node T_J41,  the nearest transportation node from P_L336.
Repair P_L345: The power crew 3 is at T_J8 at t = 100.0 minutes. It takes 14 minutes to reach nearest node T_J27,  the nearest transportation node from P_L345.
Repair W_PMA834: The water crew 1 is at T_J8 at t = 100.0  minutes. It takes 11 minutes to reach nearest node T_J7, the nearest transportation  node from W_PMA834.
Repair W_PMA833: The water crew 2 is

In [16]:
print(f"Total travel times: power: {network_recovery.power_crew_total_tt} mins, water: {network_recovery.water_crew_total_tt} min, transportation: {network_recovery.transpo_crew_total_tt} mins")

Total travel times: power: 53 mins, water: 106 min, transportation: 34 mins


In [17]:
bf_simulation.network_recovery.event_table.to_csv("event_tbl.csv", sep = "\t", index = False)

In [18]:
bf_simulation.network_recovery.get_event_table().head()

,time_stamp,components,perf_level,component_state
0,0,W_PMA687,100,Functional
11,0,T_L168,100,Functional
10,0,T_L37,100,Functional
9,0,P_L345,100,Functional
7,0,P_L377,100,Functional


In [19]:
bf_simulation.expand_event_table(1)

### (e) Simulation of interdependent effects

In [20]:
resilience_metrics = bf_simulation.simulate_interdependent_effects(
    bf_simulation.network_recovery)

Simulating network conditions at 60 s
Simulation time:  60.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 60.0 and 6060.0...


C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


******************

Simulating network conditions at 6060 s
Simulation time:  6060.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 6060.0 and 6780.0...
******************

Simulating network conditions at 6780 s
Simulation time:  6780.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 6780.0 and 6900.0...
******************

Simulating network conditions at 6900 s
Simulation time:  6900.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 6900.0 and 7020.0...
******************

Simulating network conditions at 7020 s
Simulation time:  7020.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 7020.0 and 7140.0...
******************

Simulating network conditions at 7140 s
Simulation time:  7140.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Up

### (f) Calculation of resilience metric

In [125]:
resilience_metrics.calculate_power_resmetric(network_recovery)

The Resilience Metric value based on ECS is 0.162 equivalent outage hours (EOH)
The Resilience Metric value based on PCS is 0.244 equivalent outage hours (EOH)


In [126]:
resilience_metrics.calculate_water_resmetrics(network_recovery)

The Resilience Metric value based on ECS is 5.196 equivalent outage hours (EOH)
The Resilience Metric value based on PCS is 5.806 equivalent outage hours (EOH)


End of the notebook